In [52]:
#generic linear programming
from ortools.linear_solver import pywraplp
Obj_Func = ""
sign = 3

def getConstrExpr(constraint):
    if "<=" in constraint:
        splitted_constraint = constraint.split("<=")

    elif ">=" in constraint:
        splitted_constraint = constraint.split(">=")

    lhs = splitted_constraint[0]
    rhs = splitted_constraint[1]

    splitted_lhs = []
    if "+" in lhs:
        splitted_lhs = lhs.split("+")

    elif "-" in lhs:
        splitted_lhs = lhs.split("-")
    
    else:
        splitted_lhs.append(lhs)

    #print("splitted lhs:",splitted_lhs)
    
    num_of_var = len(splitted_lhs)
    
    new_constr = ""
    for nv in range(0,num_of_var):
        coeff_index = splitted_lhs[nv].split('x')
        coeff = coeff_index[0]
        index = coeff_index[1]
        if coeff=="":
            coeff = "1"
        new_constr+=str(coeff)+" * x["+str(index)+"]"
        if nv!=num_of_var-1:
            if "+" in lhs:
                new_constr+=" + "
            elif "-" in lhs:
                new_constr+=" - "
          
    if "<=" in constraint:
        new_constr+=" <= "+str(rhs)

    elif ">=" in constraint:
        new_constr+=" >= "+str(rhs)
    return new_constr
  
    
def getExpr(splitted_objective_function,sign):
    obj_func = ""
    for nv in range(0,num_of_var):
        coeff_index = splitted_objective_function[nv].split('x')
        coeff = coeff_index[0]
        index = coeff_index[1]
        if coeff=="":
            coeff = "1"
        obj_func+=str(coeff)+" * x["+str(index)+"]"
        if nv!=num_of_var-1:
            #if sign==1:
            #    obj_func+=" + "
            #elif sign==0:
            #    obj_func+=" - "
            obj_func+=" + "
    
    return obj_func

def main():
    # [START solver]
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver('GLOP')
    # [END solver]

    # [START variables]
    infinity = solver.infinity()
    
    # Define the decision variables
    command = "Maximize 8x1+5x2"
    splitted_command = command.split()
    todo = splitted_command[0]

    if "+" in splitted_command[1]:
        sign = 1
        splitted_objective_function = splitted_command[1].split("+")

    elif "-" in splitted_command[1]:
        sign = 0
        splitted_objective_function = splitted_command[1].split("-")

    num_of_var = len(splitted_objective_function)
    
    x = {i: solver.NumVar(0.0, infinity, f'x{i}') for i in range(1, num_of_var+1)}

    print('Number of variables =', solver.NumVariables())
    # [END variables]

    # [START constraints]
    # 2x+y<=1000
    constr_list = ["2x1+x2<=1000","3x1+4x2<=2400","x1+x2<=700","x1-x2<=350","x1>=0","x2>=0"]
    for constr in constr_list:
        constr = getConstrExpr(constr)
        constr = eval(constr)
        solver.Add(constr)

    # x >=0
    #solver.Add(x[1] >= 0.0)

    # y >=0
    #solver.Add(x[2] >= 0.0)

    print('Number of constraints =', solver.NumConstraints())
    # [END constraints]
    # [START objective]
    # Maximize 8x+5y
    Obj_Func = getExpr(splitted_objective_function,sign)
    #converting string into expr
    Obj_Func = eval(Obj_Func)
    
    if todo=="Maximize":
        #solver.Maximize(8 * x[1] + 5 * x[2])
        solver.Maximize(Obj_Func)
    # [END objective]

    # [START solve]
    status = solver.Solve()

    # [END solve]

    # [START print_solution]
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        print('x1 =', x[1].solution_value())
        print('x2 =', x[2].solution_value())
        print("offset ", solver.Objective().BestBound())
    else:
        print('The problem does not have an optimal solution.')
    # [END print_solution]
    print("cons",solver.constraints()[2].ub())
    # x=round(x.solution_value())
    constraint_compute=solver.ComputeConstraintActivities()

    compute_slack=[]
    for i in range(len(solver.constraints())):
        # print("poss ", solver.constraints()[i].ub()-round(constraint_compute[i]))
        if solver.constraints()[i].ub()==infinity:
            print("inf")
        else:
            compute_slack.append( {"constraint":solver.constraints()[i],"slack": solver.constraints()[i].ub()-round(constraint_compute[i])})
    # [START advanced]
    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    # [END advanced]

    ############# slack/surplus  --------------------------------

    for val in compute_slack:
        # print(val['constraint'])
        if val['slack'] ==0.0:
            print("This constraint is a binding constraint")
            print("the constraint ", val['constraint'].GetCoefficient(x[1]),"* x1 +",val['constraint'].GetCoefficient(x[2]) ,"* x2 has a direct impact on the optimal solution")
            print("shadow price is ", val['constraint'].dual_value())
        else:
            print("This constraint is a non-binding constraint", val['constraint'])
            print("dual ", val['constraint'].dual_value())
            print("the constraint ", val['constraint'].GetCoefficient(x[1]),"* x1 +",val['constraint'].GetCoefficient(x[2]) ,"* x2 has no direct impact on the optimal solution")
            if  val['slack'] >0.0:
                print("The total limit is not used yet. It had total limit of", val['constraint'].ub(), "but used only",  val['constraint'].ub()-val['slack'])
            else:
                print("The total limit is not used yet. It had total limit of", val['constraint'].ub(), "but used only",  val['constraint'].ub()+val['slack'])

    print("the reduced cost of x1 is", x[1].reduced_cost(), x[2].basis_status())
    #print("the reduced cost of y is", y.reduced_cost(), x.basis_status())


def checkSign(val):
    if val <0:
        return True
    else:
        return False

if __name__ == '__main__':
    main()
# [END program]

2
Number of variables = 2
Number of constraints = 6
Solution:
Objective value = 4360.0
x1 = 319.99999999999983
x2 = 360.00000000000017
offset  -inf
cons 700.0
inf
inf

Advanced usage:
Problem solved in 3.000000 milliseconds
Problem solved in 4 iterations
Problem solved in -1 branch-and-bound nodes
This constraint is a binding constraint
the constraint  2.0 * x1 + 1.0 * x2 has a direct impact on the optimal solution
shadow price is  3.399999999999999
This constraint is a binding constraint
the constraint  3.0 * x1 + 4.0 * x2 has a direct impact on the optimal solution
shadow price is  0.4000000000000004
This constraint is a non-binding constraint <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x00000201B17B31E0> >
dual  -0.0
the constraint  1.0 * x1 + 1.0 * x2 has no direct impact on the optimal solution
The total limit is not used yet. It had total limit of 700.0 but used only 680.0
This constraint is a non-binding con